In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import timeit
import numba
from numba import jit, njit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
@njit
def source(r, th):

    # s, sigma, Pc = params

    # f = sigma*s*(P - Pc)**(2*sigma-1)*np.heaviside(P - Pc, 0)

    f = np.sin(th)*(1/np.tan(th)**2-1)/r**2 

    return f
@njit
def source_1(r, u):

    f1 = u*(1/np.tan(th)**2-1)/r**2
    
    return f1

@njit
def u_analytical(r, th):

    uan = np.sin(th) 

    return uan



In [4]:
def grad_shaf_solver(matrix, source, source1, grid, init_guess=None, 
                    boundary=((0, 0), (0,0)), tolerance=1.e-8, 
                    itermax=1000, omega=1.5, params=(1, 1, 0)):

    M, f, f1, u0, B, tol, kmax = matrix, source, source1, init_guess, boundary, tolerance, itermax

    r, th = grid

    R, TH = r[0], th[:,0]

    Nr, Nth = r.shape[1]-1, th.shape[0]-1

    dr, dth = (r[0, -1] - r[0, 0])/Nr, (th[-1, 0] - th[0, 0])/Nth

    if init_guess is None:

        u0 = np.ones_like(f)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                               
    u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    #initial values before the iteration loop starts
    u = u0.copy()
    u1 = u0.copy()
    k = 0
    rel_diff = tol + 1
    conv_hist = []


    #iteration loop using nested loops.
    while  k < kmax and rel_diff > tol:    

        u_next = u.copy()
        u1_next = u1.copy()

        f1 = source_1(r,u1)
        # print(f1)

        for j in range(1, Nth):
            for i in range(1, Nr):

                u_next[j,i] = (1-omega)*u[j,i] + omega/(2*(R[i]**2*dth**2 + dr**2))*(R[i]**2*dth**2*(u[j,i+1] + u_next[j, i-1]) + 
                                                                                      dr**2*(u[j+1,i]*(1+dth/(2*np.tan(TH[j]))) + 
                                                                                     u_next[j-1,i]*(1 - dth/(2*np.tan(TH[j])))) 
                                                                                     -f[j,i]*dr**2*dth**2*R[i]**2)
                #-------------------------------------------------------------------------------------------------------------
                
                u1_next[j,i] = (1-omega)*u1[j,i] + omega/(2*(R[i]**2*dth**2 + dr**2))*(R[i]**2*dth**2*(u1[j,i+1] + u1_next[j, i-1]) + 
                                                                                      dr**2*(u1[j+1,i]*(1+dth/(2*np.tan(TH[j]))) + 
                                                                                     u1_next[j-1,i]*(1 - dth/(2*np.tan(TH[j])))) 
                                                                                     -f1[j,i]*dr**2*dth**2*R[i]**2)
                                                                                

        rel_diff = la.norm(u_next-u)/la.norm(u)
        
        conv_hist.append(rel_diff)

        u = u_next
        u1 = u1_next

        k += 1

    return u, u1, k, rel_diff, conv_hist

In [5]:
#Setup the parameters for the solver. Functions grid(), source() are in the basic.py module

R, TH, r, th, dr, dth = polar_grid(th_I=0., th_F=np.pi, r_I=0, r_F=1, Nr=40, Nth=40)                     #The grid of the problem

u_guess = np.ones(r.shape)       #initial guess

s = 1.6
sigma = 1.
Pc = 0.5
params = (s , sigma, Pc)

f = source(r, th)
f1 = source_1(r,u_guess)
print(f1)
                                #source term
u_an = u_analytical(r, th)

# boundary = ((0,np.sin(np.pi*np.cos(TH))*np.sin(np.pi*np.sin(TH))+np.cos(TH)), (-R, -R))              #boundary conditions
boundary = ((np.sin(TH), np.sin(TH)), (0., 0.))

iterations = 2000                              #max number of iteration
tolerance = 1.e-17                               #desired tolerance

omega_opt = 2/(1+np.sin(np.pi*max(dr, dth)))     #relaxation parameter for SOR method

[[     inf      inf      inf ...      inf      inf      inf]
 [     inf 2.57e+05 6.42e+04 ... 1.78e+02 1.69e+02 1.60e+02]
 [     inf 6.22e+04 1.55e+04 ... 4.31e+01 4.09e+01 3.89e+01]
 ...
 [     inf 6.22e+04 1.55e+04 ... 4.31e+01 4.09e+01 3.89e+01]
 [     inf 2.57e+05 6.42e+04 ... 1.78e+02 1.69e+02 1.60e+02]
 [     inf 1.07e+35 2.67e+34 ... 7.39e+31 7.01e+31 6.67e+31]]


In [6]:
grad_shaf_solver = jit(nopython=True)(grad_shaf_solver)

In [7]:
start = timeit.default_timer()

u, u1, k, rel_diff, conv_hist = grad_shaf_solver(np.eye(99), f, f1, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

elapsed = timeit.default_timer() - start

error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

SOR solver 
Number of iterations: 2000 
Last Relative difference: 7.772E-16
Error to analytical: 4.594E-04s 
Elapsed time: 1.83E+01s 
--------------------------------------------------


In [8]:
#Plot results. Function plot is in basic.py module.
plot(r*np.cos(th), r*np.sin(th), u, u_an=u_an, conv_hist=conv_hist)

In [16]:
np.allclose(u, u1, 1.e-2)
# print(u, u1)

True

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

#Scatter plot of  u
scat = ax.scatter(r*np.cos(th), r*np.sin(th), u, c = 'r', label = 'numerical', alpha = 1)
scat1 = ax.scatter(r*np.cos(th), r*np.sin(th), u1, c = 'g', label = 'numerical1', alpha = 1)
ax.scatter(r*np.cos(th), r*np.sin(th), np.abs(u-u1))
#Surface plot of analytical solution if provided
# if u_an is not None:
    
#     surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.8)
#     #Commands to fix a bug in the legend
#     surf._facecolors2d=surf._facecolors3d
#     surf._edgecolors2d=surf._edgecolors3d

#Make plot nice and beautifull
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y + x$')

ax.legend(loc = 2)